In [1]:
import os, matplotlib.pyplot as plt, numpy as np, pandas as pd
import seaborn as sns#; sns.set()
from glob import glob
from datetime import datetime
from common_functions import load_data_stoichometry, generate_figures_and_xls, load_data_train_test_val, generate_figures_and_xls_all_strains

## benchmarking of methods and features for varying stoichiometries (Fig S5C)

In [2]:
nn = 1 # how many neighbour positions to take into account
dt_shift = 10 # expected shift between center of the pore and motor protein in bases
features = ["si", "tr", "dt0", "dt%s"%dt_shift]

fracs = np.arange(0.0, 1.01, 0.2); print(fracs)

guppy_ver = "3.0.3.hac" #"4.0.15.hac"# 
fasta = "cc_yeast_rrna.fa" # reference FastA
fnpat = "guppy%s/RNA814001_*/workspace/*.bam"%guppy_ver # pattern for all BAM files

# load data from first KO experiment
regions = [("25s", 2129, "snR3"), ("25s", 2133, "snR3"), ("25s", 2264, "snR3"), ("25s", 2826, "snR34"), ("25s", 2880, "snR34"), ("18s", 1187, "snR36"), 
          ]
bams = sorted(glob(fnpat))
samples1 = [fn.split(os.path.sep)[-3].split("_")[-1] for fn in bams]; print(samples1, regions)
region2data = load_data_stoichometry(fasta, bams, regions, features, samples1, fracs, nn=nn)
#'''
# load data from second KO experiment
fnpat2 = "guppy%s/RNA345944_*/workspace/*.bam"%guppy_ver
regions2 = [('25s', 1133, 'snR61'), ('25s', 1888, 'snR62'), ('25s', 817, 'snR60'), ('25s', 908, 'snR60'), 
#            ('25s', 1437, ''), ('25s', 867, ''), 
           ]
bams2 = sorted(glob(fnpat2))
samples2 = [fn.split(os.path.sep)[-3].split("_")[-1] for fn in bams2]; print(samples2, regions2)
region2data.update(load_data_stoichometry(fasta, bams2, regions2, features, samples2, fracs, nn=nn))
#'''
# define features
feature_names = ["%s_%s"%(f.upper(), i) for f in features for i in range(-nn, nn+1)]
len(feature_names), len(region2data)
samples = ["unmod0", "unmod", "mod",]+list(map(str, np.round(fracs, 2)))

[0.  0.2 0.4 0.6 0.8 1. ]
['snR3', 'snR34', 'snR36', 'wt'] [('25s', 2129, 'snR3'), ('25s', 2133, 'snR3'), ('25s', 2264, 'snR3'), ('25s', 2826, 'snR34'), ('25s', 2880, 'snR34'), ('18s', 1187, 'snR36')]
{'snR3': 0, 'snR34': 1, 'snR36': 2, 'wt': 3}


['snR60', 'snR61', 'snR62', 'wt'] [('25s', 1133, 'snR61'), ('25s', 1888, 'snR62'), ('25s', 817, 'snR60'), ('25s', 908, 'snR60')]
{'snR60': 0, 'snR61': 1, 'snR62': 2, 'wt': 3}


## estimation of modification frequency across all rRNA wt and mutants (Fig 3J)

In [3]:
outdir = "results/rRNA.per_method" # output directory
ext = "pdf" # ext for figures
if not os.path.isdir(outdir): 
    os.makedirs(outdir)

group2pos = {"negative": ["25s:1004", "25s:986", "25s:1437", "25s:867"], 
             "pU": ['18s:1187', '25s:2129', '25s:2133', '25s:2264', '25s:2826', '25s:2880'], 
             "Nm": ['25s:1133', '25s:1888', '25s:817', '25s:908'],
            }
cols_starts = [("SI",), ("TR",), ("DT0",), #("DT10",), # individual features
               ("SI", "TR"), ("SI", "DT0",), #("SI", "DT10"), # two type of features
               #("SI", "TR", "DT0"), #("SI", "TR", "DT10"),
              ]
# start xls writer to _tables.xlsx
xls = pd.ExcelWriter(os.path.join(outdir, '_tables.xlsx'))
freqs = generate_figures_and_xls(outdir, cols_starts, region2data, ext, xls, group2pos, feature_names, samples)    
xls.close()

GMM+eIF
GMM
AggClust
KMeans
OCSVM
IF
eIF
KNN
RF


In [4]:
region2data2 = load_data_train_test_val(fasta, bams, regions, features, samples1, nn=nn)
region2data2.update(load_data_train_test_val(fasta, bams2, regions2, features, samples2, nn=nn))

{'snR3': 0, 'snR34': 1, 'snR36': 2, 'wt': 3}


{'snR60': 0, 'snR61': 1, 'snR62': 2, 'wt': 3}


In [5]:
outdir = "results/rRNA.per_method.per_strain"
if not os.path.isdir(outdir): os.makedirs(outdir)
group2pos = {#"negative": ["25s:1004", "25s:986", "25s:1437", "25s:867"], 
             "pU": ['18s:1187', '25s:2129', '25s:2133', '25s:2264', '25s:2826', '25s:2880'], 
             "Nm": ['25s:1133', '25s:1888', '25s:817', '25s:908'],
            }
cols_starts = [("SI", "TR"), ("SI", "DT0"), ("SI", ), ("TR", ), ]
xls = pd.ExcelWriter(os.path.join(outdir, '_tables.xlsx'))
all_freqs = generate_figures_and_xls_all_strains(outdir, cols_starts, region2data2, ext, xls, group2pos, feature_names, ["unmod0", "unmod", "mod",]+samples1)
xls.close()

KMeans.SI_TR
KMeans.SI_DT0
KMeans.SI
KMeans.TR
KNN.SI_TR
KNN.SI_DT0
KNN.SI
KNN.TR
GMM.SI_TR
GMM.SI_DT0
GMM.SI
GMM.TR
AggClust.SI_TR
AggClust.SI_DT0
AggClust.SI
AggClust.TR
IF.SI_TR
IF.SI_DT0
IF.SI
IF.TR
RF.SI_TR
RF.SI_DT0
RF.SI
RF.TR
